In [1]:
# Read in Library
import pandas as pd
import pandas as pd
from ta import add_all_ta_features
import call_Strategies 

      vix_close w5_8_13_signal mfi_signal eom_signal cmf_signal Signal  \
0           NaN              0          0          0          0      0   
1     12.330000              0          0    bullish          0      0   
2     12.250000              0          0    bullish          0      0   
3     12.120000              0          0    bullish          0      0   
4     12.290000              0          0    bullish          0      0   
...         ...            ...        ...        ...        ...    ...   
7911        NaN            NaN        NaN        NaN        NaN    NaN   
7912  17.459999            NaN        NaN        NaN        NaN    NaN   
7913  17.940001            NaN        NaN        NaN        NaN    NaN   
7914  15.650000            NaN        NaN        NaN        NaN    NaN   
7915  14.600000            NaN        NaN        NaN        NaN    NaN   

     trend_strength cmf_signal 5_8_13_signal ma_direction  ... roc_signal  \
0                 0          0    

Read in SPY OHLCV Data from 10/29/1993 to 06/02/2023

In [2]:
# Read in Price Data
csv_file = 'SPY.csv'
stock_df = pd.read_csv(csv_file)

Read in Underlining Stock Financial data like 
- Dividend rate 
- P/E
- Market Cap
- 

In [3]:
# Stock Financial Data


Load Indicators

In [4]:
# Indicators
indicators_df = pd.DataFrame(index=stock_df.index)

# Add all technical indicators using TA library
indicators_df = add_all_ta_features(
    stock_df, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=False
)


c:\Users\zebfr\anaconda3\lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
c:\Users\zebfr\anaconda3\lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


Load all Strategy signals. 
- bullish/bearish
- buy/sell
- overbought/oversold
- strong/weak (trend)

In [5]:
# Strategy Signals
all_signals_df = call_Strategies.generate_all_signals('SPY.csv', 'VIX.csv')
print(all_signals_df)


      vix_close w5_8_13_signal mfi_signal eom_signal cmf_signal Signal  \
0           NaN              0          0          0          0      0   
1     12.330000              0          0    bullish          0      0   
2     12.250000              0          0    bullish          0      0   
3     12.120000              0          0    bullish          0      0   
4     12.290000              0          0    bullish          0      0   
...         ...            ...        ...        ...        ...    ...   
7911        NaN            NaN        NaN        NaN        NaN    NaN   
7912  17.459999            NaN        NaN        NaN        NaN    NaN   
7913  17.940001            NaN        NaN        NaN        NaN    NaN   
7914  15.650000            NaN        NaN        NaN        NaN    NaN   
7915  14.600000            NaN        NaN        NaN        NaN    NaN   

     trend_strength cmf_signal 5_8_13_signal ma_direction  ... roc_signal  \
0                 0          0    

Load True signals

True Signals is a csv of the most optimal buy and sell points and close of position in SPY from 01/29/1993 to 06/02/2023. It consists of a 'signals' and 'close_position' column. "long" in the 'signals' column reprsents opening a buy poistion and "short" represents opening a sell position. A 1 in the 'close_position' column represents closing any current open position (buy or sell). It is backtested in the true signals folder.

It is important to read conditions of 'close_position' column first so that you close your current position before reading the 'signals' column and opening a new one

In [10]:
# True Signals
true_signals_df = pd.read_csv("./true_signals/SPY_true_signals.csv")
print(true_signals_df)

            Date       Close signals  close_position 
0     1993-01-29   43.937500    long              NaN
1     1993-02-01   44.250000     NaN              NaN
2     1993-02-02   44.343750     NaN              NaN
3     1993-02-03   44.812500     NaN              NaN
4     1993-02-04   45.000000     NaN              1.0
...          ...         ...     ...              ...
7636  2023-05-26  420.019989     NaN              NaN
7637  2023-05-30  420.179993     NaN              NaN
7638  2023-05-31  417.850006     NaN              NaN
7639  2023-06-01  421.820007     NaN              NaN
7640  2023-06-02  427.920013     NaN              NaN

[7641 rows x 4 columns]


Merge all Dataframes

In [9]:
# Pre-process Data
merge_df = pd.concat([indicators_df, all_signals_df, true_signals_df], axis = 1)

In [ ]:
# Models


In [ ]:
# Predict Buy/Sell Condition


In [ ]:
# Evaluate
